## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [1]:
from arcgis.gis import GIS

gis = GIS("home")

#### Now you are ready to start!

In [2]:
import pandas as pd
import arcgis
import json
import requests
from arcgis.apps.dashboard import Gauge, MapLegend, NumberSelector, PieChart, RichText, SidePanel,Indicator, List, SerialChart, CategorySelector, DatePicker, Details, EmbeddedContent
from arcgis.apps.dashboard import Dashboard, add_column, add_row
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from time import sleep

In [3]:
nr_stacji = pd.read_excel('/arcgis/home/nr_stacji.xlsx')

In [4]:
#respond_st_0=requests.get(f"https://api.gios.gov.pl/pjp-api/v1/rest/metadata/sensors?page=0&size=500")
#data_st_0=json.loads(respond_st_0.text)
#stanowiska=pd.DataFrame(data_st_0['Lista metadanych stanowisk pomiarowych'])

#respond_sta_0=requests.get(f"https://api.gios.gov.pl/pjp-api/v1/rest/metadata/stations?page=0&size=500")
#data_sta_0=json.loads(respond_sta_0.text)
#stacje=pd.DataFrame(data_sta_0['Lista metadanych stacji pomiarowych'])

#for i in range(1,23):
#    respond_st=requests.get(f"https://api.gios.gov.pl/pjp-api/v1/rest/metadata/sensors?page={i}&size=500")
#    data_st=json.loads(respond_st.text)
#    try:
#        data_st['Lista metadanych stanowisk pomiarowych']
#    except KeyError:
#        exists = False
#    else:
#        d_st=pd.DataFrame(data_st['Lista metadanych stanowisk pomiarowych'])
#        stanowiska=pd.concat([stanowiska,d_st])
#to nie git

#for i in range(1,3):
#    respond_sta=requests.get(f"https://api.gios.gov.pl/pjp-api/v1/rest/metadata/stations?page={i}&size=500")
#    data_sta=json.loads(respond_sta.text)
#    try:
#        data_sta['Lista metadanych stacji pomiarowych']
#    except KeyError:
#        exists = False
#    else:
#        d_sta=pd.DataFrame(data_sta['Lista metadanych stacji pomiarowych'])
#        stacje=pd.concat([stacje,d_sta])
#to nie git

In [5]:
stanowiska_1 = pd.read_excel('/arcgis/home/stanowiska_1.xlsx')

In [6]:
stacje_1 = pd.read_excel('/arcgis/home/stacje_1.xlsx')

In [7]:
columns=['Kod stanowiska','Data','Wartość']
d_pm10= pd.DataFrame(columns=columns)
d_pm25= pd.DataFrame(columns=columns)
d_cdm= pd.DataFrame(columns=columns)
#d_x=pd.DataFrame(columns=columns)


In [8]:
#testy
#k=51
#responsen = requests.get(f"https://api.gios.gov.pl/pjp-api/v1/rest/data/getData/{k+1}")
#data_temn=json.loads(responsen.text)
#d_x3n=pd.DataFrame(columns=columns)
#dfxn=pd.DataFrame(data_temn['Lista danych pomiarowych'])
#print(dfxn["Wartość"][1])
#d_x3n=pd.concat([d_x3n, dfxn.take([1])])


In [9]:
for i in range(5522):
    #sleep(3)
    try:   
        response = requests.get(f"https://api.gios.gov.pl/pjp-api/v1/rest/data/getData/{i+1}")
        data_temp=json.loads(response.text)
        try:
            data_temp['Lista danych pomiarowych']
        except KeyError:
            exists = False
        else:
            if (data_temp['Lista danych pomiarowych'] is not None):
                if(data_temp['Lista danych pomiarowych']!=[]):
                    df=pd.DataFrame(data_temp['Lista danych pomiarowych'])
                    if(df['Wartość'][0]!=None):
                        if("-PM10-" in df['Kod stanowiska'][0]):
                            d_pm10=pd.concat([d_pm10, df.take([0])])
                        elif("-PM2.5-" in df['Kod stanowiska'][0]):
                            d_pm25=pd.concat([d_pm25, df.take([0])])
                        elif("-O3-" in df['Kod stanowiska'][0] ):
                            d_cdm=pd.concat([d_cdm, df.take([0])])
    except requests.exceptions.ConnectionError:
        exists = False


In [10]:
#usuwanie  duplikatów i wierszy z NA
d_cdm=d_cdm.drop_duplicates().dropna()
d_pm10=d_pm10.drop_duplicates().dropna()
d_pm25=d_pm25.drop_duplicates().dropna()


In [11]:
#łączenie dataframe stanowisk i stacji w jedna ramke
df_new=pd.merge(stanowiska_1,stacje_1,how="left",on=["Kod stacji"])
df_new_3=df_new[["Kod stanowiska","WGS84 φ N","WGS84 λ E","Nazwa strefy","Typ obszaru","Miejscowość"]]
df_new_3=df_new_3.rename(columns={"WGS84 φ N":"y","WGS84 λ E":"x"})




In [12]:
#łączenie wyników z ramką z współrzędnymi
df_new_pm10=pd.merge(d_pm10,df_new_3,how="left",on=["Kod stanowiska"])
df_new_pm25=pd.merge(d_pm25,df_new_3,how="left",on=["Kod stanowiska"])
df_new_cdm=pd.merge(d_cdm,df_new_3,how="left",on=["Kod stanowiska"])


In [13]:
#export do csv 
#d_p10=df_p10.to_csv(index=False)
#d_m_stanowiska = df_m_stanowiska.to_csv(index=False)
#d_m_stacje = df_m_stacje.to_csv(index=False)
#len(d_p10)
#len(d_m_stanowiska)
#df_p10
cdm_csv=df_new_cdm.to_csv("/arcgis/home/cdm.csv")
pm10_csv=df_new_pm10.to_csv("/arcgis/home/pm10.csv")
pm25_csv=df_new_pm25.to_csv("/arcgis/home/pm25.csv")

In [14]:
df_new_pm10

,Kod stanowiska,Data,Wartość,y,x,Nazwa strefy,Typ obszaru,Miejscowość
0,DsKlodzSzkol-PM10-1g,2024-01-08 09:00:00,36.13820,50.433493,16.653660,strefa dolnośląska,miejski,Kłodzko
1,KpGrudPilsud-PM10-1g,2024-01-08 09:00:00,36.80000,53.493550,18.762139,strefa kujawsko - pomorska,miejski,Grudziądz
2,KpToruWSikor-PM10-1g,2024-01-08 09:00:00,26.70000,53.012475,18.605681,miasto Toruń,miejski,Toruń
3,KpBydPlPozna-PM10-1g,2024-01-08 09:00:00,24.20000,53.121764,17.987906,Aglomeracja Bydgoska,miejski,Bydgoszcz
4,KpBydWarszaw-PM10-1g,2024-01-08 09:00:00,29.30000,53.134083,17.995708,Aglomeracja Bydgoska,miejski,Bydgoszcz
5,KpToruDziewu-PM10-1g,2024-01-08 09:00:00,26.90000,53.028647,18.666103,miasto Toruń,miejski,Toruń
6,KpToruKaszow-PM10-1g,2024-01-08 09:00:00,37.30000,53.017628,18.612808,miasto Toruń,miejski,Toruń
7,KpWloclOkrze-PM10-1g,2024-01-08 09:00:00,32.20000,52.658467,19.059314,miasto Włocławek,miejski,Włocławek
8,LbLubObywate-PM10-1g,2024-01-08 09:00:00,21.70000,51.259431,22.569133,Aglomeracja Lubelska,miejski,Lublin
9,LdGajewUjWod-PM10-1g,2024-01-08 09:00:00,28.00000,52.143258,19.233217,strefa łódzka,pozamiejski,Gajew


In [15]:
from arcgis.features import FeatureSet,FeatureLayerCollection
from arcgis.mapping import WebMap

In [16]:
#tworzenie obiektu klasy WebMap z mapa projektowa
#wm_item = gis.content.search('mapa_projekt_2')[0]
#wwm = WebMap(wm_item)


In [17]:
#tworzenie 3 FLC na bazie pobranych danych (tylko 1. raz)
'''
pm10_properties3 = {
  "title": "pm10_p3",
  "description": "pm10 imported from CSV file",
  "tags": "pm10_p3"
}

csv3_file = 'home/pm10.csv'
csv3_item = gis.content.add(pm10_properties3, csv3_file)
pm10_p3=csv3_item.publish()
pm10_flc3=FeatureLayerCollection.fromitem(pm10_p3)



o3_properties = {
  "title": "o3_p3",
  "description": "o3 imported from CSV file",
  "tags": "o3_p3"
}

csv3_file1 = 'home/cdm.csv'
csv3_item1 = gis.content.add(o3_properties, csv3_file1)
o3_p3=csv3_item1.publish()
o3_p3_flc=FeatureLayerCollection.fromitem(o3_p3)

pm25_properties3 = {
  "title": "pm25_p3",
  "description": "pm25 imported from CSV file",
  "tags": "pm25_p3"
}

csv3_file2 = 'home/pm25.csv'
csv3_item2 = gis.content.add(pm25_properties3, csv3_file2)
pm25_p3=csv3_item2.publish()
pm25_flc3=FeatureLayerCollection.fromitem(pm25_p3)
'''

'\npm10_properties3 = {\n  "title": "pm10_p3",\n  "description": "pm10 imported from CSV file",\n  "tags": "pm10_p3"\n}\n\ncsv3_file = \'home/pm10.csv\'\ncsv3_item = gis.content.add(pm10_properties3, csv3_file)\npm10_p3=csv3_item.publish()\npm10_flc3=FeatureLayerCollection.fromitem(pm10_p3)\n\n\n\no3_properties = {\n  "title": "o3_p3",\n  "description": "o3 imported from CSV file",\n  "tags": "o3_p3"\n}\n\ncsv3_file1 = \'home/cdm.csv\'\ncsv3_item1 = gis.content.add(o3_properties, csv3_file1)\no3_p3=csv3_item1.publish()\no3_p3_flc=FeatureLayerCollection.fromitem(o3_p3)\n\npm25_properties3 = {\n  "title": "pm25_p3",\n  "description": "pm25 imported from CSV file",\n  "tags": "pm25_p3"\n}\n\ncsv3_file2 = \'home/pm25.csv\'\ncsv3_item2 = gis.content.add(pm25_properties3, csv3_file2)\npm25_p3=csv3_item2.publish()\npm25_flc3=FeatureLayerCollection.fromitem(pm25_p3)\n'

In [18]:
#dodawanie warstw do mapy (tylko 1. raz)
'''
wm.add_layer(o3_p3_flc)
wm.add_layer(pm25_flc3)
wm.add_layer(pm10_flc3)
'''

'\nwm.add_layer(o3_p3_flc)\nwm.add_layer(pm25_flc3)\nwm.add_layer(pm10_flc3)\n'

In [19]:
#updatowanie danych w FLC o nowopobrane dane
FeatureLayerCollection.fromitem(gis.content.get('5978a7c3c56140c7a63a0d52670831d7')).manager.overwrite('/arcgis/home/pm10.csv')
FeatureLayerCollection.fromitem(gis.content.get('5295d605747c4304a605c92b94850233')).manager.overwrite('/arcgis/home/pm25.csv')
FeatureLayerCollection.fromitem(gis.content.get('f284acf6faa543818a682c8e976041f7')).manager.overwrite('/arcgis/home/cdm.csv')

{'success': True}

In [20]:
#wm.update()
#updatowanie mapy (tylko raz)